In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona
import rasterio
import rasterio.mask
import os
from shapely.geometry import Point
import numpy as np

In [114]:
hackberry_polygons_grid = gpd.read_file('hackberry8x8.shp')

In [115]:
hackberry_polygons_grid

,id,left,top,right,bottom,geometry
0,2021284.0,620214.0,3349143.6,620218.8,3349138.8,"POLYGON ((620218.072 3349138.846, 620218.572 3..."
1,2021285.0,620214.0,3349138.8,620218.8,3349134.0,"POLYGON ((620218.316 3349138.343, 620218.319 3..."
2,2021287.0,620214.0,3349129.2,620218.8,3349124.4,"POLYGON ((620218.685 3349127.590, 620218.681 3..."
3,2022838.0,620218.8,3349143.6,620223.6,3349138.8,"POLYGON ((620219.075 3349139.084, 620219.078 3..."
4,2022839.0,620218.8,3349138.8,620223.6,3349134.0,"POLYGON ((620222.013 3349134.048, 620221.763 3..."
...,...,...,...,...,...,...
3472,2046058.0,620290.8,3349575.6,620295.6,3349570.8,"POLYGON ((620295.404 3349573.403, 620295.407 3..."
3473,2047611.0,620295.6,3349580.4,620300.4,3349575.6,"POLYGON ((620296.230 3349579.642, 620296.227 3..."
3474,2047612.0,620295.6,3349575.6,620300.4,3349570.8,"POLYGON ((620299.643 3349572.601, 620299.640 3..."
3475,2049165.0,620300.4,3349580.4,620305.2,3349575.6,"POLYGON ((620300.707 3349577.838, 620300.704 3..."


In [121]:
# hackberry_polygons_grid_proj = hackberry_polygons_grid.to_crs({'init': 'epsg:3857'})
# hackberry_polygons_grid_proj.crs

In [116]:
hackberry_polygons_grid["area"] = hackberry_polygons_grid['geometry'].area/ 10**6

In [117]:
hackberry_polygons_grid

,id,left,top,right,bottom,geometry,area
0,2021284.0,620214.0,3349143.6,620218.8,3349138.8,"POLYGON ((620218.072 3349138.846, 620218.572 3...",8.680588e-08
1,2021285.0,620214.0,3349138.8,620218.8,3349134.0,"POLYGON ((620218.316 3349138.343, 620218.319 3...",2.727003e-07
2,2021287.0,620214.0,3349129.2,620218.8,3349124.4,"POLYGON ((620218.685 3349127.590, 620218.681 3...",8.220553e-07
3,2022838.0,620218.8,3349143.6,620223.6,3349138.8,"POLYGON ((620219.075 3349139.084, 620219.078 3...",7.449289e-06
4,2022839.0,620218.8,3349138.8,620223.6,3349134.0,"POLYGON ((620222.013 3349134.048, 620221.763 3...",1.493002e-05
...,...,...,...,...,...,...,...
3472,2046058.0,620290.8,3349575.6,620295.6,3349570.8,"POLYGON ((620295.404 3349573.403, 620295.407 3...",1.593304e-06
3473,2047611.0,620295.6,3349580.4,620300.4,3349575.6,"POLYGON ((620296.230 3349579.642, 620296.227 3...",1.498202e-05
3474,2047612.0,620295.6,3349575.6,620300.4,3349570.8,"POLYGON ((620299.643 3349572.601, 620299.640 3...",1.451065e-05
3475,2049165.0,620300.4,3349580.4,620305.2,3349575.6,"POLYGON ((620300.707 3349577.838, 620300.704 3...",2.901616e-06


In [119]:
area8x8 = hackberry_polygons_grid['area'].max()
area8x8*0.3

6.911999999798834e-06

In [120]:
hackberry_polygons_grid[hackberry_polygons_grid['area'] >= area8x8*0.3]

,id,left,top,right,bottom,geometry,area
3,2022838.0,620218.8,3349143.6,620223.6,3349138.8,"POLYGON ((620219.075 3349139.084, 620219.078 3...",0.000007
4,2022839.0,620218.8,3349138.8,620223.6,3349134.0,"POLYGON ((620222.013 3349134.048, 620221.763 3...",0.000015
5,2022840.0,620218.8,3349134.0,620223.6,3349129.2,"POLYGON ((620219.209 3349129.583, 620219.459 3...",0.000012
6,2022841.0,620218.8,3349129.2,620223.6,3349124.4,"POLYGON ((620223.397 3349124.532, 620223.395 3...",0.000016
8,2024392.0,620223.6,3349143.6,620228.4,3349138.8,"POLYGON ((620227.116 3349142.485, 620227.113 3...",0.000015
...,...,...,...,...,...,...,...
3465,2047340.0,620295.6,3350881.2,620300.4,3350876.4,"POLYGON ((620295.764 3350880.798, 620295.761 3...",0.000011
3466,2047341.0,620295.6,3350876.4,620300.4,3350871.6,"POLYGON ((620299.705 3350876.001, 620299.702 3...",0.000014
3471,2046057.0,620290.8,3349580.4,620295.6,3349575.6,"POLYGON ((620295.227 3349579.405, 620295.230 3...",0.000014
3473,2047611.0,620295.6,3349580.4,620300.4,3349575.6,"POLYGON ((620296.230 3349579.642, 620296.227 3...",0.000015


In [122]:
hackberry_rect = hackberry_polygons_grid[hackberry_polygons_grid['area']>=area8x8*0.3]
hackberry_rect.crs

<Derived Projected CRS: EPSG:26914>
Name: NAD83 / UTM zone 14N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 102°W and 96°W - onshore and offshore. Canada - Manitoba; Nunavut; Saskatchewan. United States (USA) - Iowa; Kansas; Minnesota; Nebraska; North Dakota; Oklahoma; South Dakota; Texas.
- bounds: (-102.0, 25.83, -96.0, 84.0)
Coordinate Operation:
- name: UTM zone 14N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [123]:
hackberry_rect.to_file('hackberry_rect8x8.shp')

In [28]:
file_names = ['test_se_raster']

for file_name in file_names:
    # clip the raster file with polygons
    # read polygons_species_mode shapefile
    with fiona.open('test_rect4x4.shp', 'r') as shapefile:
        shapes = [feature['geometry'] for feature in shapefile]

    # read imagery file
    raster_file = file_name + str('.tif')
    with rasterio.open(raster_file) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta

    # save clipped imagery
    out_meta.update({'driver': 'GTiff', 
                     'height': out_image.shape[1], 
                     'width': out_image.shape[2], 
                     'transform': out_transform})

    clipped_file_name = file_name + str('_clipped.tif')
    with rasterio.open(clipped_file_name, 'w', **out_meta) as dest: 
        dest.write(out_image)

/Users/kennywu/opt/anaconda3/lib/python3.8/site-packages/rasterio/mask.py:191: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(


## Test single species

In [111]:
combine_polygons = gpd.read_file('combine_polygons_26914.shp')

In [32]:
combine_polygons['SPECIES'].value_counts()[:10]

pecan                  1759
elm cedar              1591
southern live oak      1426
escarpment live oak     964
crape myrtle            423
hackberry               321
post oak                310
ashe juniper            232
shumard oak             223
elm american            188
Name: SPECIES, dtype: int64

In [99]:
pecan_polygons = combine_polygons[combine_polygons['SPECIES']=='pecan']

In [101]:
pecan_polygons.to_file('pecan_polygons.shp')

In [85]:
ashe_polygons = combine_polygons[combine_polygons['SPECIES']=='ashe juniper']

In [87]:
ashe_polygons.to_file('ashe_polygons.shp')

In [113]:
hackberry_polygons = combine_polygons[combine_polygons['SPECIES']=='hackberry']
hackberry_polygons.to_file('hackberry_polygons.shp')

In [36]:
combine_8x8grid = gpd.read_file('combine_8x8grid.shp')

In [ ]:
se_texture = gpd.sjoin(hackberry_polygons, combine_8x8grid, how='inner', op='within')
se_texture.drop(columns='index_right',inplace=True)

In [135]:
hackberry_1500 = rasterio.open('hackberry/hackberry_1500.tif').read()

In [139]:
hackberry_514 = rasterio.open('hackberry/hackberry_514.tif').read()
hackberry_10 = rasterio.open('hackberry/hackberry_10.tif').read()

In [140]:
hackberry_10.shape

(4, 9, 9)

In [138]:
hackberry_1500.shape

(4, 9, 8)

In [132]:
hackberry_514.shape

(4, 9, 9)

In [137]:
hackberry_1500

array([[[ 43.,  51.,  49.,  42.,   0.,   0.,   0.,   0.],
        [ 41.,  44.,  51.,  56.,   0.,   0.,   0.,   0.],
        [ 43.,  41.,  42.,  53.,  61.,   0.,   0.,   0.],
        [ 55.,  52.,  46.,  46.,  56.,  60.,   0.,   0.],
        [ 65.,  64.,  58.,  48.,  47.,   0.,   0.,   0.],
        [ 70.,  68.,  65.,  55.,   0.,   0.,   0.,   0.],
        [ 67.,  63.,  56.,  50.,  53.,  64.,   0.,   0.],
        [ 69.,  67.,  63.,  66.,   0.,   0.,  65.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]],

       [[ 50.,  63.,  65.,  58.,   0.,   0.,   0.,   0.],
        [ 52.,  55.,  63.,  70.,   0.,   0.,   0.,   0.],
        [ 55.,  54.,  53.,  67.,  76.,   0.,   0.,   0.],
        [ 71.,  67.,  59.,  58.,  71.,  76.,   0.,   0.],
        [ 81.,  79.,  74.,  59.,  59.,   0.,   0.,   0.],
        [ 89.,  85.,  82.,  69.,   0.,   0.,   0.,   0.],
        [ 84.,  79.,  72.,  72.,  64.,  75.,   0.,   0.],
        [ 83.,  88.,  81.,  81.,   0.,   0.,  80.,   0.],
        [  0

In [133]:
hackberry_514

array([[[ 66.,  63.,  71.,  65.,  67.,  66.,  81.,  81.,   0.],
        [ 70.,  67.,  78.,  65.,  56.,  49.,  69.,  74.,   0.],
        [ 74.,  78.,  84.,  74.,  55.,  43.,  48.,  58.,   0.],
        [ 78.,  78.,  83.,  92.,  74.,  60.,  53.,  53.,   0.],
        [ 79.,  82.,  75.,  91.,  84.,  79.,  67.,  67.,   0.],
        [ 75.,  78.,  86.,  77.,  71.,  78.,  77.,  73.,   0.],
        [ 80.,  78.,  89.,  79.,  74.,  73.,  69.,  70.,   0.],
        [ 71.,  78.,  85.,  84.,  78.,  79.,  79.,  74.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]],

       [[ 80.,  73.,  78.,  76.,  85.,  81.,  96.,  92.,   0.],
        [ 83.,  78.,  88.,  76.,  68.,  60.,  77.,  82.,   0.],
        [ 86.,  87.,  93.,  88.,  65.,  57.,  59.,  68.,   0.],
        [ 92.,  89.,  96., 101.,  85.,  72.,  68.,  66.,   0.],
        [ 98.,  99.,  90., 103.,  99.,  89.,  80.,  79.,   0.],
        [ 92.,  90.,  94.,  87.,  83.,  93.,  90.,  86.,   0.],
        [ 93.,  89., 102.,  91.,  84.,